In [ ]:
from load_data import *
from load_reg_data import *
from battery import *
import pandas as pd
import os
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import datetime

In [ ]:
#dates, freq, master_df = load_data_func(os, pd, np)

area = 'SE4'

freq, master_df = load_data_func(os, pd, np, area)
dates = master_df.index

In [ ]:
rev_list_2021 = []

rev_list_2021.append(sum(master_df['FCR-N Volume'][:8760]*master_df['FCR-N Price'][:8760]))
rev_list_2021.append(sum(master_df['FCR-D up Volume'][:8760]*master_df['FCR-D up Price'][:8760]))
rev_list_2021.append(sum(master_df['FCR-D down Volume'][:8760]*master_df['FCR-D down Price'][:8760]))
rev_list_2021.append(sum(master_df['aFRR up Volume'][:8760]*master_df['aFRR up Price'][:8760]))
rev_list_2021.append(sum(master_df['aFRR down Volume'][:8760]*master_df['aFRR down Price'][:8760]))
rev_list_2021.append(sum(rev_list_2021))

rev_list_2021[:] = [round(x / 1000000)for x in rev_list_2021]

rev_list_2022 = []

rev_list_2022.append(sum(master_df['FCR-N Volume'][8760:]*master_df['FCR-N Price'][8760:]))
rev_list_2022.append(sum(master_df['FCR-D up Volume'][8760:]*master_df['FCR-D up Price'][8760:]))
rev_list_2022.append(sum(master_df['FCR-D down Volume'][8760:]*master_df['FCR-D down Price'][8760:]))
rev_list_2022.append(sum(master_df['aFRR up Volume'][8760:]*master_df['aFRR up Price'][8760:]))
rev_list_2022.append(sum(master_df['aFRR down Volume'][8760:]*master_df['aFRR down Price'][8760:]))
rev_list_2022.append(sum(rev_list_2022))

rev_list_2022[:] = [round(x / 1000000)for x in rev_list_2022]

print(rev_list_2021)
print(rev_list_2022)

In [ ]:
print(round(np.mean(master_df['FCR-N Volume'][8760:])),
    round(np.mean(master_df['FCR-D up Volume'][8760:])),
    round(np.mean(master_df['FCR-D down Volume'][8760:])),
    round(np.mean(master_df['aFRR up Volume'][8760:])),
    round(np.mean(master_df['aFRR down Volume'][8760:])))

In [ ]:
fix, ax = plt.subplots()

ax.plot(freq[8760*60*6:-1*60*6:30])

plt.show()

In [ ]:
def interval_finder(start_year, start_month, start_day, start_hour, start_nbr, end_year, end_month, end_day, end_hour, end_nbr):
    start_time = datetime.datetime(start_year, start_month, start_day, start_hour, start_nbr)
    end_time = datetime.datetime(end_year, end_month, end_day, end_hour, end_nbr)
    start_idx = int(np.where(dates == start_time)[0])
    end_idx = int(np.where(dates == end_time)[0])
    return range(start_idx, end_idx)

In [ ]:
def activation_percentage(service_name, freq_vec, freq_ind):

    '''
    The method will return the level of activated power for any given frequency. It takes into account which service is activated.
    Postive activation power means adding power to the grid, negative means subtracting power from the grid.

    Return the share of activated power. [-1, 1]

    # Remember that the frequency is one hour behind/forward in Finland were the frequency is sampled.
    
    '''

    activ = 0
    curr_freq = freq_vec[freq_ind]

    freq_moving_average_available = False
    freq_moving_average = 0

    if freq_ind > 8760*360 + 5:

        freq_moving_average_available = True
        freq_moving_average = np.mean(freq_vec[freq_ind - 6:freq_ind])

    if service_name == 'FFR': # For now we're calling this as 0 always. It is not significant energy throughput anyways.
        activ = 0

    if service_name == 'FCR-N':

        activ = 1 # Base case

        if 49.9 < curr_freq < 50.1:
            activ = (50.0 - curr_freq) / 0.1
        if curr_freq >= 50.1:
            activ = -1

    if service_name == 'FCR-D up':

        if  curr_freq < 49.9:
            activ = min(1, (49.9 - curr_freq) / 0.4)

    if service_name == 'FCR-D down':

        if  curr_freq > 50.1:
            activ = max(-1, (50.1 - curr_freq) / 0.4)

    # Be careful with the assumptions made below.
    '''
    if service_name == 'aFRR up':
        
        # If
        if curr_freq < 50.0 and freq_vec[freq_ind - 1] < 50.0:
            activ = min(1, (50.0 - curr_freq) / 0.5)
        
    if service_name == 'aFRR down':

        if curr_freq > 50.0 and freq_vec[freq_ind - 1] > 50.0:
            activ = max(-1, (50.0 - curr_freq) / 0.5)
    
    '''

    if service_name == 'aFRR up':

        if freq_moving_average_available: # Given that we are in the start of the sequence we only use the current frequency.

            if freq_moving_average < 50.0: 
                activ = min(1, (50.0 - freq_moving_average) / 0.1)

        else:
                        
            if curr_freq < 50.0: 
                activ = min(1, (50.0 - curr_freq) / 0.1)
        
    if service_name == 'aFRR down':

        if freq_moving_average_available: # Given that we are in the start of the sequence we only use the current frequency.

            if freq_moving_average > 50.0: 
                activ = max(-1, (50.0 - freq_moving_average) / 0.1)

        else:
                        
            if curr_freq > 50.0: 
                activ = max(-1, (50.0 - curr_freq) / 0.1)

    return activ

In [ ]:
def h_pay_and_cycles (service_name, battery, hour, freq_vec, df, sample_freq):

    '''
    Returns the sum of the capacity cash flow and the activation cash flow as well as the number of cycles run.
    '''
    did_run = False
    price_str = service_name + ' Price'
    counts_per_hour = int(3600*sample_freq)
    
# Remember that the frequency is one hour behind/forward in Finland were the frequency is sampled.

    # Change later
    spot = df['Elspot']
    up_reg_price = df['Up Reg Price']
    down_reg_price = df['Down Reg Price']

    act_list = []
    capacity_cash_flow = 0
    activation_cash_flow = np.zeros(counts_per_hour)

    if hour % 2 == 0: # We may bid every other hour.
        did_run = True
        capacity_cash_flow = battery.storage_capacity * df[price_str][hour]
        
        for count in range(counts_per_hour):

            required_energy = abs(battery.max_power * activation_percentage(service_name, freq_vec, hour*counts_per_hour + count) / (3600*sample_freq))
            act_list.append(required_energy)

            up_reg = True
            if freq_vec[hour + count] > 50.00:
                up_reg = False

            if service_name == 'FCR-D up' or service_name == 'FCR-D down':
                
                if up_reg:
                    activation_cash_flow[count] = -spot[hour] * required_energy
                else:
                    activation_cash_flow[count] = spot[hour] * required_energy # If we down regulate we are paid by the spot price...

            else:
            
                if up_reg:
                    activation_cash_flow[count] = up_reg_price[hour] * required_energy
                else:
                    activation_cash_flow[count] = down_reg_price[hour] * required_energy
                    
    cycles = sum(act_list) / (battery.storage_capacity * 2)
    cash_flow = capacity_cash_flow + sum(activation_cash_flow)

    return cash_flow, cycles, did_run

In [ ]:
#def energy_arb (df, battery, interval):

battery = Battery()
spot = master_df['Elspot'][8760:]
nbr_days = 365
earnings = 0
round_trip_eff = 0.85
earnings = []
for day in range(nbr_days):
    spot_this_day = spot[day:day+24]
    daily_min = min(spot_this_day)
    daily_max = max(spot_this_day)
    diff = daily_max - daily_min
    earnings.append(battery.storage_capacity*diff*round_trip_eff)

print(sum(earnings))
    
#    return earnings

In [ ]:
'''
Outputs a DataFrame of how any single frequency regulating service would have performed during 2022. IRR, Pay-back being the central KPIs.
'''

services = ['FFR', 'FCR-N', 'FCR-D up', 'FCR-D down', 'aFRR up', 'aFRR down']
sample_freq = 1/10
battery = Battery()
investment = 6.5e7
info = pd.DataFrame()
economic_life_span = 1
deg_cost = 21000 # Cost per MWh (i.e. cost/MWh) We're counting in SEK.
#opex_per_mw_and_hour = 30 # Might have to change too
opex = 0.01*investment

interval = interval_finder(2022, 1, 1, 0, 0, 2022, 12, 31, 22, 0) # The last data point to use is 22.00. That is 23.00 in Finland.

for service in services:

    earnings = np.zeros(len(interval))
    cycles = np.zeros(len(interval))

    for count, hour in enumerate(interval):

        earnings[count], cycles[count], did_run = h_pay_and_cycles(service, battery, hour, freq, master_df, sample_freq)

    y_cycles = sum(cycles)
    y_earning = sum(earnings)

    deg_costs = deg_cost*y_cycles
    #opex = 8760*battery.max_power*opex_per_mw_and_hour

    economic_life_span = min()

    pay_back = investment/(y_earning - opex - deg_costs)
    irr = npf.irr([-investment] + economic_life_span*[y_earning - opex - deg_costs])

    new_dict = {'Service': service, 'Revenue': y_earning, '# Cycles': y_cycles, 'Degradation cost': deg_costs, 'Pay-back': pay_back}
    # 'IRR': irr*100,
    new_df = pd.DataFrame(new_dict, index=[0])
    info = info.append(new_df, ignore_index=True)

print(info.round(1).to_latex())

In [ ]:
'''
SE1

{} &     Service &     Revenue &  \# Cycles &  Degradation cost &  Pay-back \\
\midrule
0 &         FFR &   9228420.0 &       0.0 &               0.0 &      10.6 \\
1 &       FCR-N &  29701557.9 &   13783.3 &               0.0 &       2.6 \\
2 &    FCR-D up &  28122880.6 &      18.2 &               0.0 &       2.7 \\
3 &  FCR-D down &  14207403.7 &      23.8 &               0.0 &       6.0 \\
4 &     aFRR up &  32184051.7 &    6699.7 &               0.0 &       2.4 \\
5 &   aFRR down &  23477338.3 &    7083.7 &               0.0 &       3.4 \\
\bottomrule
\end{tabular}

------------------

 &     Service &     Revenue &  \# Cycles &  Degradation cost &  Pay-back \\
0 &         FFR &   9228420.0 &       0.0 &               0.0 &      10.6 \\
1 &       FCR-N &  29750666.6 &   13783.3 &               0.0 &       2.6 \\
2 &    FCR-D up &  28122353.5 &      18.2 &               0.0 &       2.7 \\
3 &  FCR-D down &  14207152.1 &      23.8 &               0.0 &       6.0 \\
4 &     aFRR up &  32426876.0 &    6699.7 &               0.0 &       2.3 \\
5 &   aFRR down &  23599476.9 &    7083.7 &               0.0 &       3.3 \\


'''

'''
Outputs a DataFrame consiting of the optimized bidding for 2022. For every hour, the most profitable bid is chosen. The performance of the bidding strategy is summarized in KPIs as before.
''' 


In [33]:
# What should we do in this part of the data analysis? 
# We should go through all hours, note the availability earnings, activation earings, degradation cost, and total cash flow for that hour
def optimized_run (master_df, freq, area_name, interest_rate):
    
    services = ['FFR', 'FCR-N', 'FCR-D up', 'FCR-D down', 'aFRR up', 'aFRR down']
    sample_freq = 1/10
    battery = Battery()
    investment = 6.5e7
    info = pd.DataFrame()
    economic_life_span = 0
    deg_cost = 21000 # Cost per MWh (i.e. cost/MWh) We're counting in SEK.
    #opex_per_mw_and_hour = 30 # Might have to change too
    opex = 0.01*investment

    interval = interval_finder(2022, 1, 1, 0, 0, 2022, 1, 2, 2, 0) # The last data point to use is 22.00. That is 23.00 in Finland.

    best_choice_list = []
    best_choice_df = pd.DataFrame()
    tot_cycles = 0
    tot_earnings = 0
    tot_degradation_costs = 0

    for hour in interval:

        best_choice = 'Pause'
        best_choice_idx = len(services)




        if hour % 2 == 0:

            service_hour_profit = []
            cycles = np.zeros(len(services) + 1) # Leave room for the case that none of the services were chosen
            earnings = np.zeros(len(services) + 1)
            degradation_cost = np.zeros(len(services) + 1)

            for count, service in enumerate(services):

                earnings[count], cycles[count], did_run = h_pay_and_cycles(service, battery, hour, freq, master_df, sample_freq)
                degradation_cost[count] = cycles[count]*deg_cost*battery.storage_capacity

                service_hour_profit.append(earnings[count] - degradation_cost[count])

            best_choice_idx = service_hour_profit.index(max(service_hour_profit))
            best_choice = services[best_choice_idx] # Will provide of that hour's most profitable service choice

            tot_cycles = tot_cycles + cycles[best_choice_idx]
            tot_earnings = tot_earnings + earnings[best_choice_idx]
            tot_degradation_costs = tot_degradation_costs + degradation_cost[best_choice_idx]

        #dict_to_append = {'Service': best_choice, 'Earnings': earnings[best_choice_idx], 'Cycles': cycles[best_choice_idx], 'Degradation costs':  degradation_cost[best_choice_idx] , 'Hourly profit': max(service_hour_profit)}
        #best_choice_df = best_choice_df.append(dict_to_append, ignore_index= True)
        best_choice_list.append(best_choice)

    economic_life_span = min([15, round(3000/tot_cycles)])
    print(economic_life_span)
    tot_deg_costs = tot_cycles*deg_cost*battery.storage_capacity

    irr = npf.irr([-investment] + economic_life_span*[tot_earnings - opex - tot_deg_costs])
    npv = npf.npv(interest_rate, [-investment] + economic_life_span*[tot_earnings - opex - tot_deg_costs])

    final_dict = {'Earnings': tot_earnings, 'Cycles': tot_cycles, 'Degradation costs': tot_deg_costs, 'IRR': irr, 'Pay-back': investment/(tot_earnings - opex - tot_deg_costs), 'NPV': npv}

    sum_df = pd.DataFrame(final_dict, index=[0])
    counts = best_choice_df['Service'].value_counts()
    interest_rate_str = str(interest_rate*100)

    sum_df.to_csv(area_name + '_interest_rate_' + interest_rate_str + '_optimized_df' + '.csv')

In [34]:
areas = ['SE1', 'SE2', 'SE3', 'SE4']
interest_rate = 0.08

for count, this_area in enumerate(areas):
    freq, master_df = load_data_func(os, pd, np, this_area)
    dates = master_df.index
    optimized_run(master_df, freq, this_area, interest_rate)

15


KeyError: 'Service'

In [ ]:
''' SE1
Pause         4379
FCR-D up      3237
FCR-D down     487
aFRR up        275
FFR            250
aFRR down      129
FCR-N            1
'''

''' SE2
Pause         4379
FCR-D up      3229
FCR-D down     487
aFRR up        280
FFR            250
aFRR down      132
FCR-N            1
'''

''' SE3
Pause         4379
FCR-D up      3275
FCR-D down     491
FFR            250
aFRR up        243
aFRR down      119
FCR-N            1
'''

''' SE4
Pause         4379
FCR-D up      3274
FCR-D down     492
FFR            250
aFRR up        243
aFRR down      119
FCR-N            1
'''


In [ ]:
# Finding the most valeable service

FSR_df = master_df[['FCR-D down Price', 'FCR-D up Price', 'FCR-N Price', 'aFFR down Price', 'aFFR up Price']][8760:]

max_type = FSR_df.idxmax(axis = 1)
max_val = FSR_df.max(axis = 1)
counts = max_type.value_counts()
print(counts)

In [ ]:
start_time = datetime.datetime(2022, 1, 1, 0, 0)
end_time = datetime.datetime(2022, 12, 31, 23, 0)
start_idx = int(np.where(dates == start_time)[0])
end_idx = int(np.where(dates == end_time)[0])
interval = range(start_idx, end_idx)


arr = master_df['FCR-D up Price'].to_numpy().reshape(-1)
window_len = 24*7
moving_ave = np.zeros(len(master_df['FCR-D up Price']))
print(arr)

a = 1
b = 10
for t in range(window_len + 1, ):
    moving_ave[t] = arr[t- 1: t]

fix, ax = plt.subplots()

ax.plot(master_df['FCR-D up Price'])

plt.show()

print(type(master_df['FCR-D up Price']))

In [ ]:
start_time = datetime.datetime(2022, 1, 1, 0, 0)
end_time = datetime.datetime(2022, 12, 31, 23, 0)
start_idx = int(np.where(dates == start_time)[0])
end_idx = int(np.where(dates == end_time)[0])
interval = range(start_idx, end_idx)

fig, ax = plt.subplots()

ax.plot(master_df['FCR-D up Price'][8760:], label = 'FCR-D up Price')
ax.plot(master_df['FCR-D down Price'][8760:], label = 'FCR-D down Price')
ax.plot(master_df['FCR-N Price'][8760:], label = 'FCR-N Price')
# ax.plot(master_df['Elspot'][8760:], label = 'Spot')
# ax.plot(master_df['aFFR up Price'][8760:], label = 'aFFR up Price')
# ax.plot(master_df['aFFR down Price'][8760:], label = 'aFFR down Price')

plt.legend()
plt.title('Hourly prices')
plt.ylabel('SEK/MW')

plt.show()